# Introduction
This is the Marginal Effects at Means portion of the project
Please be sure to run DataCleaning.ipynb first to prepare the data

This notebook based on Fastai V1 ML course

## Imports
Import libraries and write settings here.

In [1]:
from fastai.tabular import *
from fastai import *

# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 60
pd.options.display.max_rows = 60

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

# DETERMINE IF YOU ARE USING RAW OR SCALED
Scaled is Zhao's data, continuous variables divided by standard deviation
<br>Raw is unscaled raw data

In [2]:
useSavedModel=True
filename_model_params = "RF_model_params.sav"
useRaw=False
outdir = 'outBElectionResultsScaled2'
filename='results.csv'
filename_all = 'results_complete.csv'
filename_model_params = "RF_model_params.sav"
os.makedirs(outdir, exist_ok=True)

# the dependant variable(s)
columns_dep_var= ['cc.TrumpGEVote','cc.TrumpPVote', 'cc.vote12.gop']

#get the data
if (useRaw == True):
    df = pd.read_csv('./data/ccesplus.csv',encoding = "ISO-8859-1")
else:
    df = pd.read_csv('./data/ccesplus_fscaled.csv',encoding = "ISO-8859-1")

## Get the columns of interest, including all dependent vars

In [3]:
all = pd.read_excel('./data/CCESplusVariables.xlsx',encoding = "ISO-8859-1")

# for every regid=3.1 get the third column (variable name) convert to a list
columns = list(all[ (all['regid']==1.1) & (all['exclude']!='t')  ].iloc[:,2])

# #strip dependant variables
# columns = [col for col in columns if col not in columns_dep_var]
# columns

In [4]:
#WARNING! 'cc.i.white.educhs not in df! No worries though 
#its correlated with composed of cc.maxeduc.hs and cc.WhiteDum
#so drop it
columns.remove('cc.i.white.educhs')

In [56]:
len(columns)
# columns

35

## Extract just those columns from orig dataframe

In [51]:
df = df[columns]

In [52]:
len(df.columns)

35

# Choose which dependant variable to operate on

In [53]:
dep_var = columns_dep_var[0]
dump_these_dep_var_columns=columns_dep_var.copy()
dump_these_dep_var_columns.remove(dep_var)

## Remove NaN dependent variable rows


In [119]:
def remove_dep_var_nan_rows(df, col_dep_var, dump_these_dep_var_columns ,silent = False):
    """
    parse df into rows where df.col_dep_var does not have NaNs, 
    df: dataframe to pull NaN rows out of
    col_dep_var: dependent variable
    dump_these_dep_var_columns: other dep_var columns that may be highly correlated with dep var
    """
    dftmp= df.copy()
    
    if(silent is False):
        print(f'dropping {(pd.isnull(dftmp[col_dep_var])).sum()} rows where {col_dep_var}=NaN')
                      
    #dump the NaN rows
    dftmp = dftmp[pd.notnull(dftmp[col_dep_var] )]
    
    #dump the dump_these_dep_var_columns
    dftmp.drop(dump_these_dep_var_columns,axis=1,inplace=True);
    
    return dftmp

# pull out the nulls from the column of interest
dftmp= remove_dep_var_nan_rows(df, columns_dep_var[0],dump_these_dep_var_columns )

dropping 23710 rows where cc.TrumpGEVote=NaN


In [120]:
#get a list of the columns to operate on minus the dependant variable
columns = dftmp.columns.drop(dep_var)
columns

Index(['cc.emp.nojob', 'cc.CC16_304', 'cc.faminc', 'crashpc', 'rustpc',
       'job.uer', 'econ.mhi', 'econ.hhpov.p', 'taa.wrks.disc95.pcpy',
       'cc.CC16_305_2', 'mort.ucd.despair.disc95.pdpy', 'cc.immviewsum',
       'cc.raceviewsum', 'cc.CC16_307', 'cc.CC16_351B', 'cc.Age', 'cc.Sex',
       'cc.BlackDum', 'cc.WhiteDum', 'cc.maxeduc.4yr', 'cc.maxeduc.hs',
       'cc.isimmigrant', 'cc.child18', 'cc.union', 'cc.newsint', 'cc.Ideo7',
       'cc.EvanProt', 'cc.Catholic', 'cc.Religiosity', 'demo.popdense',
       'cc.RepDum', 'cc.IndDum'],
      dtype='object')

## Split out categorical and continuous variables
see if fastai can auto do it

In [121]:
# #give fastai a shot at splitting cat and cont variables
res_cont, res_cat = cont_cat_split(dftmp,max_card=18)

## Categorify and Fill Missing

In [118]:
#from docs https://docs.fast.ai/tabular.transform.html
# tfm = Categorify(cat_names=res_cat, cont_names=res_cont)
# tfm(dftmp)

In [122]:
tfm1 = FillMissing(cat_names=res_cat, cont_names=res_cont, add_col=False)
tfm1(dftmp)

## Generate train and test set

In [123]:
# split out train/test sets
from sklearn.model_selection import train_test_split

trn=tst=None
trn,tst = train_test_split(dftmp, test_size=0.1)
print(str(len(trn)))
print(str(len(tst)))

36801
4089


In [124]:
len(trn.columns)

33

In [125]:
#split out trn_y and tst_y
#this is the dep_var, converted to an int
trn_y = trn[dep_var].copy()
tst_y = tst[dep_var].copy()
trn_y.astype('int64');
trn_y.astype('int64');

trn.drop(dep_var,axis=1,inplace=True);
tst.drop(dep_var,axis=1,inplace=True);

# Get the standard deviation for every column of interest
For continuous variables we need to subtract 1 standard deviation

# Train a RandomForest on all data

In [126]:
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics


if (useSavedModel):
    # load the model from disk
    m_rf = pickle.load(open(outdir+"/"+filename_model_params, 'rb'))
else:
    #create a random forest object
    m_rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, oob_score=True, max_features='auto', min_samples_leaf=5)

In [127]:

def rmse(x,y): 
    '''this and R**2 used for continuous variables'''
    return math.sqrt(((x-y)**2).mean())

def print_score(m, trn, trn_y, tst, tst_y):
    '''
    
    '''
    res = [m.score(trn, trn_y), m.score(tst, tst_y)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

def eval_accuracy(preds,targs, silent=True):
    totals = len(preds)
    matches = 0
    for x in zip(preds,targs):
        if x[0]==x[1]:
            matches+=1
    acc=100*matches/totals 
    if( silent == False):
        print(f"Got {matches} right out of {totals} samples, Accuracy is {acc} percent")
    return acc

In [128]:
#train the random forest 
m_rf.fit(trn, trn_y)
print_score(m_rf, trn, trn_y, tst, tst_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=999, verbose=0, warm_start=False)

[0.9388603570555148, 0.9107361213010516, 0.9160892367055243]


In [129]:
#just another way to see stuff
# preds1 = m_rf.predict(tst)
# eval_accuracy(preds1,tst_y,silent=False);

In [130]:
columns

Index(['cc.emp.nojob', 'cc.CC16_304', 'cc.faminc', 'crashpc', 'rustpc',
       'job.uer', 'econ.mhi', 'econ.hhpov.p', 'taa.wrks.disc95.pcpy',
       'cc.CC16_305_2', 'mort.ucd.despair.disc95.pdpy', 'cc.immviewsum',
       'cc.raceviewsum', 'cc.CC16_307', 'cc.CC16_351B', 'cc.Age', 'cc.Sex',
       'cc.BlackDum', 'cc.WhiteDum', 'cc.maxeduc.4yr', 'cc.maxeduc.hs',
       'cc.isimmigrant', 'cc.child18', 'cc.union', 'cc.newsint', 'cc.Ideo7',
       'cc.EvanProt', 'cc.Catholic', 'cc.Religiosity', 'demo.popdense',
       'cc.RepDum', 'cc.IndDum'],
      dtype='object')

# Now run MEMs on columns of interest

In [131]:
#this directory contains symlink created at command line like this
# ln -s ../Marginal_Effects_at_Means ./Marginal_Effects_at_Means
#it allows this directory to find Marginal_Effects_at_Means, a directory 1 above this one
#this dir contains a file called mem.py which contains MEMs
from Marginal_Effects_at_Means.mem import MEMs

In [132]:
all = trn.copy().append(tst.copy(), ignore_index=True)
all_y = trn_y.copy().append(tst_y.copy(), ignore_index=True)
len(all.columns)

32

In [153]:
from pandas.api.types import is_numeric_dtype
# all.dtypes
for val in all.columns:
    if (is_numeric_dtype(all[val])):
        print(f"{val}    {all[val].nunique()}")
#     if all[val].dtype == 'float':
#          print(f"{val}    {all[val].nunique()}")


cc.emp.nojob    2
cc.CC16_304    5
cc.faminc    16
crashpc    2439
rustpc    2264
job.uer    741
econ.mhi    2456
econ.hhpov.p    1000
taa.wrks.disc95.pcpy    2013
cc.CC16_305_2    2
mort.ucd.despair.disc95.pdpy    2466
cc.immviewsum    5
cc.raceviewsum    17
cc.CC16_307    4
cc.CC16_351B    2
cc.Age    78
cc.Sex    2
cc.BlackDum    2
cc.WhiteDum    2
cc.maxeduc.4yr    2
cc.maxeduc.hs    2
cc.isimmigrant    2
cc.child18    2
cc.union    2
cc.newsint    5
cc.Ideo7    7
cc.EvanProt    2
cc.Catholic    2
cc.Religiosity    15
demo.popdense    2473
cc.RepDum    2
cc.IndDum    2


In [159]:
all['cc.CC16_304'].max()
all['cc.CC16_304'].mean()
all['cc.CC16_304'].describe()

1.5799542961462802

0.9396806189703275

count    40890.000000
mean         0.939681
std          0.297727
min          0.315991
25%          0.631982
50%          0.947973
75%          0.947973
max          1.579954
Name: cc.CC16_304, dtype: float64

In [134]:
#RUN ON WHOLE DATASET OR JUST THE TestSET?  I'm thinking the whole dataset.  
# The model has some idea of how voters will vote based on the input features, lets use that knowledge
#to see what happens when we start changing variables
mems = MEMs(all)

In [74]:
len(dftmp.columns)

33

In [150]:
dftmp.dtypes

cc.TrumpGEVote                  float64
cc.emp.nojob                      int64
cc.CC16_304                     float64
cc.faminc                       float64
crashpc                         float64
rustpc                          float64
job.uer                         float64
econ.mhi                        float64
econ.hhpov.p                    float64
taa.wrks.disc95.pcpy            float64
cc.CC16_305_2                     int64
mort.ucd.despair.disc95.pdpy    float64
cc.immviewsum                   float64
cc.raceviewsum                  float64
cc.CC16_307                     float64
cc.CC16_351B                    float64
cc.Age                          float64
cc.Sex                            int64
cc.BlackDum                       int64
cc.WhiteDum                       int64
cc.maxeduc.4yr                    int64
cc.maxeduc.hs                     int64
cc.isimmigrant                    int64
cc.child18                      float64
cc.union                        float64


## TODO: 
- for each column 
    - get type 
    - for categorical variable (int32) increment to next int.
    - for float 64, continuous var, increment by 1. (each float64 column has already been divided by standard deviation.So its std-deviation is 1). 

In [166]:
for col in columns:
    print(f"for column {col} the results are {mems.getMEM_avgplusone(m_rf, col)}")

for column cc.emp.nojob the results are [(0, array([0.])), (1, array([0.]))]
 PROBLEM, STDDEV EXCEEDS MAX VAL, 1.9396806189703275 > 1.5799542961462802
for column cc.CC16_304 the results are [(0.9396806189703275, array([0.])), (1.9396806189703275, array([0.]))]
for column cc.faminc the results are [(0.9330832316116301, array([0.])), (1.9330832316116302, array([0.]))]
for column crashpc the results are [(0.8587735892548911, array([0.])), (1.8587735892548911, array([0.]))]
for column rustpc the results are [(0.7205782078407131, array([0.])), (1.7205782078407132, array([0.]))]
for column job.uer the results are [(0.9613650191084155, array([0.])), (1.9613650191084155, array([0.]))]
for column econ.mhi the results are [(0.9709426336017595, array([0.])), (1.9709426336017595, array([0.]))]
for column econ.hhpov.p the results are [(0.9132809117572355, array([0.])), (1.9132809117572354, array([0.]))]
for column taa.wrks.disc95.pcpy the results are [(0.6346131510264463, array([0.])), (1.634613151